# Assignment of building damage data to grids from municipalities

In [ ]:
%load_ext jupyter_black
import pandas as pd
import os
from pathlib import Path
pd.set_option("display.float_format", lambda x: "%.5f" % x)
input_dir = (
    Path(os.getenv("STORM_DATA_DIR")) / "analysis/02_new_model_input/02_housing_damage/input/"
)
baseline_input_dir = (
    Path(os.getenv("STORM_DATA_DIR")) / "analysis/01_baseline_model/input/"
)
output_dir = (
    Path(os.getenv("STORM_DATA_DIR")) / "analysis/02_new_model_input/02_housing_damage/output/"
)

In [ ]:
# reading in file with building damage data and adm3 in grid
adm3_perc_ingrid = pd.read_csv(
    input_dir / "Adm3_Perc_inGrid_Weight_Matrix.csv"
)
perc_build_dmg_data = pd.read_csv(
    baseline_input_dir / "combined_input_data.csv"
)
build_dmg_data = pd.read_csv(input_dir / "IMpact_data_philipines_SEP_2021.csv")
phl_north_build = pd.read_csv(
    input_dir / "phl_north_hotosm_bld_centroid_count.csv"
)
phl_south_build = pd.read_csv(
    input_dir / "phl_south_hotosm_bld_centroid_count.csv"
)
adm3_perc_ingrid.shape

In [ ]:
## Section describing the merging of the north and south buildings from HOTOSM shapefile
# merging the north and south values as the shapefiles are separate
merged_df_points = (
    phl_north_build[["id", "NUMPOINTS"]].set_index(["id"])
    + phl_south_build[["id", "NUMPOINTS"]].set_index(["id"])
).reset_index()
merged_df_points = pd.merge(
    merged_df_points, phl_north_build.drop("NUMPOINTS", axis=1), on="id"
)
merged_df_points

In [ ]:
merged_df_points.to_csv(
    input_dir / "phl_hotosm_bld_centroid_count.csv", index=False
)

In [ ]:
### Section describing the computation of the building damage percentage
# combining building damage data and grid information
merged_total_damage_df = adm3_perc_ingrid.merge(
    build_dmg_data,
    left_on="ADM3_PCODE",
    right_on="pcode",
    how="left",
)
merged_total_damage_df["Totally"].describe()

In [ ]:
merged_total_damage_df

In [ ]:
# computing % in each grid
# totally damaged
merged_total_damage_df["Totally_Damaged_bygrid"] = (
    merged_total_damage_df["Municipality Completeness"]
    * merged_total_damage_df["Totally"]
)
# partially damaged
merged_total_damage_df["Partially_Damaged_bygrid"] = (
    merged_total_damage_df["Municipality Completeness"]
    * merged_total_damage_df["Partially"]
)
# all damaged
merged_total_damage_df["All_Damaged_bygrid"] = (
    merged_total_damage_df["Municipality Completeness"]
    * merged_total_damage_df["total"]
)
# computing percentage damage
# merging with building damage data
merged_perc_damage_df = merged_df_points.merge(
    merged_total_damage_df,
    on="id",
    how="left",
)
merged_perc_damage_df

In [ ]:
# computing the percentage damage
merged_perc_damage_df["Totally_Damaged_Perc_bygrid"] = (
    merged_perc_damage_df["Totally_Damaged_bygrid"]
    / merged_perc_damage_df["NUMPOINTS"]
)
# partially damaged
merged_perc_damage_df["Partially_Damaged_Perc_bygrid"] = (
    merged_perc_damage_df["Partially_Damaged_bygrid"]
    / merged_perc_damage_df["NUMPOINTS"]
)
# all damaged
merged_perc_damage_df["All_Damaged_Perc_bygrid"] = (
    merged_perc_damage_df["All_Damaged_bygrid"]
    / merged_perc_damage_df["NUMPOINTS"]
)
merged_perc_damage_df["Totally_Damaged_Perc_bygrid"].describe()

In [ ]:
merged_perc_damage_df

In [ ]:
# writing output to CSV file
merged_perc_damage_df.to_csv(
    output_dir / "percentage_building_damage_bygrid.csv", index=False
)

In [ ]:
# Changing percentage damage to grids
# combining building damage data and grid information
perc_merged_df = adm3_perc_ingrid.merge(
    perc_build_dmg_data[["Mun_Code", "typhoon", "DAM_perc_dmg"]],
    left_on="ADM3_PCODE",
    right_on="Mun_Code",
    how="left",
)
perc_merged_df["DAM_perc_dmg"].describe()
perc_merged_df
# computing % in each grid
perc_merged_df["DAM_perc_dmg_Grid"] = (
    perc_merged_df["Municipality Completeness"]
    * perc_merged_df["DAM_perc_dmg"]
)
# writing output to CSV file
# perc_merged_df.to_csv(input_dir / "perc_building_dmg_bygrid.csv", index=False)